# Data Collection

In [1]:
# Data Collection
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/SpamClassifier/master/smsspamcollection/SMSSpamCollection',
                 sep = '\t',names=['label','message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Preprocessing

In [2]:
# Preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Stemming and Removing Stop_words

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
corpus = []
for i in range(len(df)):
  #review = nltk.word_tokenize(df['message'][i])
  review = re.sub('[^a-zA-Z]'," ",df['message'][i])
  review = review.lower()
  review = review.split()
  review = [stemmer.stem(word) for word in review if word not in stopwords.words('english')]
  review = " ".join(review)
  corpus.append(review)

In [6]:
corpus[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

#Feature Extraction

In [7]:
# creating BOW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
#max_features are top most most frequent elements
# we have to convert into array 
X = cv.fit_transform(corpus).toarray()

In [8]:
y =pd.get_dummies(df['label'],drop_first = True)
y
#y = y.iloc[:,1].values
# here .values is to convert it into array

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


#Splitting data

In [9]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20,random_state = 0)

#Model Training

In [10]:
# model training
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model = model.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Prediction

In [11]:
y_pred = model.predict(X_test)

#Model Evaluation

In [12]:
from sklearn.metrics import confusion_matrix,accuracy_score
matrix = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)

In [13]:
print(acc)
print(matrix)

0.9847533632286996
[[946   9]
 [  8 152]]


#HyperParameter Tuning

In [14]:
model.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True, 'force_alpha': 'warn'}

In [15]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
          'fit_prior': [True, False],
          'class_prior': [None, [0.1,]* len(X_train), ],
          'force_alpha': [True, False]}
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(estimator = model,param_grid = params,n_jobs = -1,cv = 5,verbose = True)

In [16]:
grid_cv = grid_cv.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/naive_bayes.py", line 779, in fit
    self._update_class_log_prior(class_prior=class_prior)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/naive_bayes.py", line 595, in _update_class_log_prior
    raise ValueError("Number of priors must match number of

In [17]:
grid_cv.best_estimator_

MultinomialNB(force_alpha=True)

In [18]:
y_pred = grid_cv.best_estimator_.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix,accuracy_score
matrix = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(acc)
print(matrix)

0.9847533632286996
[[946   9]
 [  8 152]]
